In [1]:
!pwd
!which python

import sys, os
import numpy as np
# To import pyface.py is just for setting "Qt" sip API version to 2 before matplotlib sets it to default v1 
# that is incompatible with "pyside" in package "mayavi".
import pyface.qt    
import matplotlib.pyplot as plt

sys.path.append('../')   # add path of package "electrode" to search list.
sys.path.append('../../bem/')    # add path of package "bem" to search list.

/home/wwc/bem_log/electrode/examples
/home/wwc/anaconda3/envs/ele36/bin/python


In [2]:
from electrode import System, GridElectrode

# load the electrostatics results into a electrode.System()
s = System()
for name in "DC1 DC2 DC3 DC4 DC5 RF".split():
    # r = Result.from_vtk(prefix, name)
    # r after from_result(): e ~ obj = cls(origin=origin, spacing=spacing, data=pot). 
    # A vtk file after from_vtk(): obj = cls(origin=origin, spacing=spacing, data=pot). 
    # So after using from_vtk() we jump over r and get to e.
    e = GridElectrode.from_vtk("../../bem/examples/SimpleTrap/SimpleTrap_"+name+".vtk",maxderiv=4)    # from_vtk() is classmethod()
    e.name = name
    s.append(e)
print(e,e.rf)

../electrode/transformations.py:1911: UserWarning: failed to import module _transformations
  warnings.warn("failed to import module %s" % name)


<electrode.electrode.GridElectrode object at 0x7ff5a0b8ddb0> 0.0


In [3]:
from scipy.constants import atomic_mass
s["RF"].rf = 50.
s["DC1"].dc = 8
s["DC4"].dc = 8
s["DC5"].dc = 2
xi = (0, 0.5, 1.5)
x0 = s.minimum(xi)
print("offset: %s\n"%(x0-xi))
# analyze_static(x, axis=(0, 1, 2), m=ct.atomic_mass, q=ct.elementary_charge, l=100e-6, o=2*np.pi*1e6, ions=1, log=None):
for line in s.analyze_static(x0, m=25*atomic_mass, l=40e-6, o=100e6*2*np.pi):      # u=50., # u is V rf peak voltage, but it causes error.
    print(line)

ValueError: failed, 0, Warning: Desired error not necessarily achieved due to precision loss.,      fun: 1.4374998632698492
     jac: array([ 7.53072047e-04, -9.00906625e-03,  2.73141756e-05])
 message: 'Warning: Desired error not necessarily achieved due to precision loss.'
    nfev: 36
    nhev: 7
     nit: 6
    njev: 30
  status: 2
 success: False
       x: array([0.00448776, 0.10030779, 0.89652894])

In [4]:
# This is from electrode turtorial.ipynb
# import scipy.constants as ct
# l = 30e-6 # µm length scale
# u = 20. # V rf peak voltage
# m = 25*ct.atomic_mass # ion mass
# q = 1*ct.elementary_charge # ion charge
# o = 2*np.pi*100e6 # rf frequency in rad/s
# s["RF"].rf = u*np.sqrt(q/m)/(2*l*o)
# print(s["RF"])

# x0 = s.minimum((0, 0, 1.), axis=(1, 2))

# for line in s.analyze_static(x0, axis=(1,0), m=m, q=q, l=l, o=o):
#     print(line)